In [1]:
import os
import time
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from timm import create_model
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
import copy

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

train_dataset = datasets.ImageFolder(root='C:/Users/liamcee/Documents/farbruh/fer13/train', transform=transform)
test_dataset = datasets.ImageFolder(root='C:/Users/liamcee/Documents/farbruh/fer13/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

num_classes = len(train_dataset.classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device, num_classes)

cuda 7


In [3]:
model = create_model(
    'swin_tiny_patch4_window7_224',
    pretrained=True,
    num_classes=num_classes
)
model = model.to(device)

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=4, gamma=0.1)

In [5]:
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

best_acc = 0.0
best_model_wts = copy.deepcopy(model.state_dict())

def train(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    loop = tqdm(loader, desc="Training")

    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        loop.set_postfix(loss=loss.item(), acc=100.0 * correct / total)

    epoch_loss = running_loss / len(loader)
    epoch_acc = 100.0 * correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)
    print(f"Train Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

def evaluate(model, loader, criterion, epoch):
    global best_acc, best_model_wts
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    loop = tqdm(loader, desc="Evaluating")

    with torch.no_grad():
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            loop.set_postfix(loss=loss.item(), acc=100.0 * correct / total)

    epoch_loss = running_loss / len(loader)
    epoch_acc = 100.0 * correct / total
    test_losses.append(epoch_loss)
    test_accuracies.append(epoch_acc)

    print(f"Test  Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

    if epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(best_model_wts, 'best_swin_fer13.pth')
        print(f"best model saved with accuracy: {best_acc:.2f}%")

In [6]:
epochs = 20
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    start_time = time.time()
    
    train(model, train_loader, optimizer, criterion)
    evaluate(model, test_loader, criterion, epoch)
    
    scheduler.step()

    elapsed = time.time() - start_time
    print(f"Time: {elapsed:.2f} s")


Epoch 1/20


Training: 100%|██████████| 1795/1795 [14:06<00:00,  2.12it/s, acc=52, loss=1.54]   


Train Loss: 1.2436, Accuracy: 51.98%


Evaluating: 100%|██████████| 449/449 [01:13<00:00,  6.10it/s, acc=62.1, loss=0.629]


Test  Loss: 1.0044, Accuracy: 62.12%
best model saved with accuracy: 62.12%
Time: 920.68 s

Epoch 2/20


...

Test  Loss: 1.6518, Accuracy: 68.61%
Time: 875.83 s

Epoch 20/20


Training: 100%|██████████| 1795/1795 [08:34<00:00,  3.49it/s, acc=96.7, loss=0.0116] 


Train Loss: 0.1051, Accuracy: 96.66%


Evaluating: 100%|██████████| 449/449 [00:38<00:00, 11.58it/s, acc=68.6, loss=0.1]     

Test  Loss: 1.6520, Accuracy: 68.61%
Time: 553.25 s


In [7]:
metrics = {
    'epoch': list(range(1, epochs + 1)),
    'train_loss': train_losses,
    'train_accuracy': train_accuracies,
    'test_loss': test_losses,
    'test_accuracy': test_accuracies
}

df_metrics = pd.DataFrame(metrics)

df_metrics.to_csv("swin_fer13_metrics.csv", index=False)

torch.save(model.state_dict(), 'ferswin_weights.pth')
torch.save(model, 'ferswin_full.pth')